In [2]:
import pygrib
import os
import pandas as pd
from datetime import datetime
from osgeo import gdal
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
import keras.backend as backend
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from dateutil import parser
import matplotlib.pyplot as plt
from pyhdf.SD import SD, SDC, SDS
import pyproj
from pyproj import CRS, Proj
from typing import Union
from shapely.geometry import Point, Polygon
import geopandas as gpd
from datetime import datetime, timedelta
import pickle
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [3]:
import os

# column_names = ['datetime', 'latitude', 'humidity', 'temperature', 'wind_u', 'wind_v', 'precip']
# df = pd.DataFrame(columns = column_names)

weather_dict = {}
counter = 0

def listdirs(rootdir):
    global weather_dict
    global counter
    for it in os.scandir(rootdir):
        if it.is_dir():
#             print(it.path)
            listdirs(it)
        
        else:
#             print(it, ' being added to dict')
#             print(type(it))
            grbs = pygrib.open(it.path)
            for grb in grbs:
                quad = (grb['dataDate'], grb['time'], grb['latitudes'], grb['parameterName'])
                weather_dict[quad] = [grb['values']]
                
rootdir = './Actual Weather Data'
listdirs(rootdir)

In [4]:
df = pd.DataFrame(weather_dict)
df2 = df.sort_index()
print(int(df2[20180202,0, 25.0]['Temperature']))
# print(df2)

# weather_dict[(20160202,0, 25.0)]

286


/var/folders/s9/bqp3gwh57sv7rgc2_ynsbcdm0000gn/T/ipykernel_51768/2031586107.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  print(int(df2[20180202,0, 25.0]['Temperature']))


In [5]:
def get_weather_features(labels, weather_data_df, grid_id_list, training):
    global features
    counter = 0
    for i in range(len(labels)):
        grid_id = list(labels['grid_id'])[i]
        j = grid_id_list.index(grid_id)
        location = grid_metadata['location'][j]
        tz = grid_metadata['tz'][j]
        datetime = list(labels['datetime'])[j]
        datetime_object = pd.to_datetime(
            datetime,
            format="%Y%m%dT%H:%M:%S",
            utc=True
        )
        
        
        datetime = datetime.replace('-', '')[:8]
        yesterday = (datetime_object - timedelta(hours=24)).strftime('%Y%m%d')
        
        hours = [0, 600, 1200, 1800]
        parameters = ['Relative humidity', 'Temperature', 'u-component of wind', 'v-component of wind', ]
       
        
        if location == 'Delhi':
            lat = 28.5
        elif location == 'Los Angeles (SoCAB)':
            lat = 34
        elif location == 'Taipei':
            lat = 25

        features_to_add = []
        for hour in hours:
            temp = []
            for param in parameters:
                temp.append(int(weather_data_df[(int(yesterday), hour, lat
                                            )][param]))
            features_to_add.append(np.array(temp))
            
        for hour in hours:
            temp = []
            for param in parameters:
                temp.append(int(weather_data_df[(int(datetime), hour, lat
                                            )][param]))
            features_to_add.append(np.array(temp))
        
        features.append(np.array(features_to_add))
        counter+=1
        if counter%100==0:
            print(counter)
#         print(features)
        

In [6]:
def get_features_b(labels, weather_data_df, grid_id_list):
    global features_b
    for i in range(len(labels)):
        grid_id = list(labels['grid_id'])[i]
        j = grid_id_list.index(grid_id)
        location = grid_metadata['location'][j]
        datetime = list(labels['datetime'])[j]
        
        if (datetime[4:6]) in ['01','02','03']:
            temp1 = [1,0,0,0]
        elif datetime[4:6] in ['04','05','06']:
            temp1 = [0,1,0,0]
        elif datetime[4:6] in ['07', '08', '09']:
            temp1 = [0,0,1,0]
        else:
            temp1 = [0,0,0,1]

        if location == 'Los Angeles (SoCAB)':
            temp2 = [1,0,0]
        elif location == 'Delhi':
            temp2 = [0,1,0]
        else:
            temp2 = [0,0,1]
        
        to_add = temp1 + temp2

        features_b.append(to_add)
        

In [ ]:
features = []
features_b = []

train_labels = pd.read_csv("train_labels.csv")
train_labels['Date'] = pd.to_datetime(train_labels['datetime'], format='%Y-%m-%d')
train_labels['Year'] = train_labels['Date'].dt.year

train_actual_labels = train_labels[train_labels['Year'] <= 2019].copy()
train_cross_val_labels = train_labels[train_labels['Year'] > 2019].copy()

# train_actual_labels = train_labels.head(750)
# train_cross_val_labels = train_labels.tail(250)

grid_metadata = pd.read_csv("grid_metadata.csv", index_col=0)
grid_id_list = list(grid_metadata['wkt'].keys())

get_weather_features(train_actual_labels, df2, grid_id_list, True)
get_features_b(train_actual_labels, df2, grid_id_list)

train_features = features.copy()
train_features_b = features_b.copy()

/var/folders/s9/bqp3gwh57sv7rgc2_ynsbcdm0000gn/T/ipykernel_51768/2959419267.py:35: PerformanceWarning: indexing past lexsort depth may impact performance.
  temp.append(int(weather_data_df[(int(yesterday), hour, lat
/var/folders/s9/bqp3gwh57sv7rgc2_ynsbcdm0000gn/T/ipykernel_51768/2959419267.py:42: PerformanceWarning: indexing past lexsort depth may impact performance.
  temp.append(int(weather_data_df[(int(datetime), hour, lat


In [ ]:
from keras import Input, Model
from keras.layers import concatenate

inputA = Input(shape=(8,4))
inputB = Input(shape=7)

x = GRU(128, dropout=.20)(inputA)
x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(.2)(x)
x = Dense(10, activation='relu')(x)
x = Model(inputs=inputA, outputs=x)

y = Dense(64, activation='relu')(inputB)
y = Dropout(.2)(y)
y = Dense(10, activation='relu')(y)
y = Model(inputs=inputB, outputs=y)

combined = concatenate([x.output, y.output])

z = Dense(64, activation='relu')(combined)
z = Dropout(.2)(z)
z = Dense(1, activation='linear')(z)

model = Model(inputs=[x.input, y.input], outputs=z)
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
# !pip install pydot
# !conda install graphviz
from keras.layers import Dropout, GRU, BatchNormalization
from keras.utils.vis_utils import plot_model
import visualkeras
import pydot

labels_array = np.array(train_labels.value)[0:750]

# def make_model():

#     model = Sequential()
#     model.add(Dense(13, input_dim=32, kernel_initializer='normal', activation='relu'))
# #     model.add(Dropout(.2))
#     model.add(Dense(6, kernel_initializer='normal', activation='relu'))
# #     model.add(Dropout(.2))
#     model.add(Dense(1, kernel_initializer='normal'))
#     # Compile model
#     model.compile(loss='mean_squared_error', optimizer='adam')
#     return model

# def make_model():
#     model = Sequential()
#     model.add(GRU(128, input_shape=(8, 4), dropout=.20))
#     model.add(BatchNormalization())
#     model.add(Dense(10, activation='relu'))
#     model.add(Dropout(.2))
#     model.add(Dense(1))
    
#     model.compile(loss='mean_squared_error', optimizer='adam')
#     return model


# model = make_model()
# plot_model(model)

# estimator = KerasRegressor(build_fn=make_model, nb_epoch=100, batch_size=5, verbose=0)

X = np.array(train_features)
X_b = np.array(train_features_b)
Y = np.array(labels_array)

print(X.shape,X_b.shape,Y.shape)

model.fit(x=[X, X_b], y=Y, epochs=100, batch_size=50)


In [ ]:
#Validating results


train_cross_val_labels = train_labels.tail(250)
# features = []
features_b = []

# get_weather_features(train_cross_val_labels, df2, grid_id_list, True)
# validation_features = features.copy()

get_features_b(train_cross_val_labels, df2, grid_id_list)
validation_features_b = features_b.copy()

predictions_2019 = model.predict([np.array(validation_features), np.array(validation_features_b)])

print(mean_squared_error(np.array(train_cross_val_labels.value), predictions_2019))
print(r2_score(train_cross_val_labels.value, predictions_2019))

In [ ]:
#Debugging why the model isn't performing as it should

for i in range(len(predictions_2019)):
    print(i, predictions_2019[i], list(train_cross_val_labels.value)[i], 
          grid_metadata['location'][list(train_cross_val_labels['grid_id'])[i]], 
         list(train_cross_val_labels['datetime'])[i])
# print(predictions_2019, train_cross_val_labels.value)

In [ ]:
#Getting predictions
features = []

train_labels = pd.read_csv("submission_format.csv") # Smallest subset
grid_metadata = pd.read_csv("grid_metadata.csv", index_col=0)

grid_id_list = list(grid_metadata['wkt'].keys())

get_weather_features(train_labels, df2, grid_id_list, True)